# Lesson 4: Node Management

<b><u>Instructors</b></u>: Ishan Mishra, Phil Culliton


In this lesson, we'll pull back the curtain a bit and show you how to manage, maintain and customize your node. We'll be walking you through adding new users, changing priviledges, creating and responding to requests, and everything to do with network nodes!


<hr>

## Concept 1. Logging in! 


### 1.1 Via PySyft
There are two ways to log into your own node, as the Data Owner. The first way is using the PySyft library:

In [1]:
import syft as sy
import numpy as np

In [2]:
domain_node = sy.login(email="info@openmined.org", password="changethis", port=8081)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to http://localhost:8081... done! 	 Logging into adp... done!



### 1.2 Via the UI

The second way involves using the same username and password, but logging in through the UI, which is accessible at <a href="http://localhost:8081">localhost:8081</a>

Please note, if you launched Hagrid to a different port, please replace 8081 with the appropriate portnumber!

![alt text](ui_images/login_screen.png "UI Login Screen")

<hr>

## Concept 2. Users!

You might not be the only person using your domain node! Let's see how you can go about checking all the users on your domain node, as well as how to add and configure new ones!

### 2.1 Seeing all users
To see all the current users on a domain node, you can simply do the following once you've signed in using the PySyft library:

In [3]:
# See all the users
domain_node.users

,added_by,allocated_budget,budget,budget_spent,created_at,daa_pdf,email,id,institution,name,role,verify_key,website
0,<syft.lib.python._SyNone object at 0x7fbb154c3...,0.0,5.55,0.0,2021-11-03 02:48:47.495871,<syft.lib.python._SyNone object at 0x7fbb154c3...,info@openmined.org,1,<syft.lib.python._SyNone object at 0x7fbb154c3...,Jane Doe,Owner,dd784cd6c0278a9cd11b69cdf4c4ca0076e661e3dd2e74...,<syft.lib.python._SyNone object at 0x7fbb154c3...


<br>

Alternatively, if you logged in using the UI, you are able to see the Users by clicking the "Users" field.

![alt text](ui_images/users.png)

<hr>

### 2.2 Creating a New User!

Creating a new user is again a really simple process. We just need to specify the following parameters when we're creating the account. Note that many of these can be changed and updated retroactively!

Let's now try creating a user via PySyft, and validating that it worked.

In [4]:
# Create a new user
domain_node.users.create(
    **{
        "name": "Sheldon Cooper",
        "email": "sheldon@caltech.edu",
        "password": "bazinga",
        "budget": 100
    }
)

In [5]:
domain_node.users

,added_by,allocated_budget,budget,budget_spent,created_at,daa_pdf,email,id,institution,name,role,verify_key,website
0,<syft.lib.python._SyNone object at 0x7fbb154c3...,0.0,5.55,0.0,2021-11-03 02:48:47.495871,<syft.lib.python._SyNone object at 0x7fbb154c3...,info@openmined.org,1,<syft.lib.python._SyNone object at 0x7fbb154c3...,Jane Doe,Owner,dd784cd6c0278a9cd11b69cdf4c4ca0076e661e3dd2e74...,<syft.lib.python._SyNone object at 0x7fbb154c3...
1,Jane Doe,0.0,100.00,0.0,2021-11-03 06:24:10.657167,1,sheldon@caltech.edu,2,,Sheldon Cooper,Data Scientist,2fef564b79c4fef21d1549a891d624d84e71c482af0c07...,


By default, this new user is a Data Scientist. However, other roles are also possible. You can see all the roles that are available by accessing <a href="http://localhost:8081/permissions"> the UI</a> and clicking on "Permissions" in the Menu bar on the left side of the screen.

<hr>

### 2.3 User Permissions

It's understandable that you might want to control how much a given user can change and tinker with your domain node. To allow for this, we've added customizable user permissions!

As mentioned previously, these are accessible by accessing the <a href="http://localhost:8081/permissions"> the UI</a> and clicking on "Permissions" in the Menu bar on the left side of the screen.

![alt text](ui_images/permissions_unexpanded.png)

Each of the 4 roles can be configured as you see fit. The parameters you can adjust are shown below:

![alt text](ui_images/permissions_all.png)

<hr>

### Concept 3. Requests

Requests are instances where a User wants to do something that they don't have authorization to do.


As of the current release, there are 3 major types of requests:
- <b> Data Access</b> requests: Occurs when someone wanting to use the data on the node.
- <b>Privacy budget upgrade</b> requests: Occurs when a DS needs an upgrade of the privacy budget allocated to them.
- <b>New Data Scientist Account</b> requests: Occurs when a Data Scientist wishes to create an account on the domain node.

With each type of request, they can either be accepted, rejected, approved or granted. We'll show you how to do each, but first, let's show you how a user would put in a request.

### 3.1 Putting in a Request

Remember that data scientist account we created awhile back? We'll be making use of him now. Let's first use him to sign into our domain node.

In [13]:
data_scientist_node = sy.login(email="sheldon@caltech.edu", password="bazinga", port=8081)

Connecting to http://localhost:8081... done! 	 Logging into adp... done!


Now that he's logged in, let's say he checks the privacy budget available to him:

In [15]:
data_scientist_node.privacy_budget

100.0

Now let's say our data scientist isn't happy with this much privacy budget, and that he wants more. He could put in a request for more privacy budget, as follows:

In [14]:
data_scientist_node.request_budget(eps=1000, reason="I want to do data exploration")

Requested 1000 epsilon of budget. Call .privacy_budget to see if your budget has arrived!


### 3.2 Responding to a Request

The request that the Data Scientist put in would be logged in the domain node, and could be viewed by the Data Owner at any time:

In [16]:
# See all the requests made to the domain node
domain_node.requests

,Requested Object's tags,Reason,Request ID,Requested Object's ID,Requested Object's type
0,[],I want to do data exploration,<UID: 3ece116558bf41cfb4680847173b6c41>,<UID: 6e2fecb392834b6589f616b7c19cd72f>,<Budget>


Now, the Data Owner could investigate this request:

In [19]:
domain_node.requests[0].request_description

'I want to do data exploration'

Let's say he decides to turn it down due to how much privacy budget is requested. He would run this:

In [60]:
domain_node.requests[0].deny()

In [32]:
domain_node.requests[0].reject()

If instead he wanted to accept the request, he could instead run:

In [33]:
domain_node.requests[0].accept()

It's important to note that after each grant or refusal, the request disappears from the object store and is no longer visible.

![alt text](ui_images/no_requests.png)

<hr>

## Concept 4. Network Nodes!

A Network Node is a level of abstraction above a domain node. It's a server which exists outside of any data owner's institution, providing services to a network of data owners and data scientists, such as dataset searching and bulk project approval (the ability to participate in projects across groups of domains and data scientsts at a time).

We can view the available networks, and put in a request to join them via the UI:

![alt text](ui_images/networks.png)

We can also do this via PySyft. To see the list of available networks we could join, we simply need to do:

In [40]:
sy.networks

,name,host_or_ip,vpn_host_or_ip,protocol,port,admin_email,website,slack,slack_channel
0,omnet,13.64.187.229,100.64.0.1,http,80,support@openmined.org,https://www.openmined.org/,https://slack.openmined.org/,omnet
1,United Nations PET Lab (Mock),13.64.14.221,100.64.0.1,http,80,support@openmined.org,https://www.openmined.org/,https://slack.openmined.org/,petlab


To join one, we would call:

In [41]:
un_network = sy.networks[1]

Connecting to http://13.64.14.221... done! 	 Logging into united_nations_mock... as GUEST...done!


<NetworkClient: united_nations_mock>

A few things to note here- 
- We joined the Network as a Guest. This has limited capabilities.
- Instead of indexing by its position in the networks, we could also index by name

- <b> Upon selecting a network, we have access to all the public properties of the network!</b>

After joining the network node, we can see all the domains that are attached to that network:

In [19]:
opengrid = sy.networks[0]
opengrid

		     opengrid network

,host_or_ip,id,is_vpn,name
0,100.64.0.3,84a28770a17d4b6199d78d66b3bdc8c3,1,relaxed_song
1,100.64.0.1,a2f53454633f4896852aae45d7bc4096,1,strange_he
2,100.64.0.4,201e5de97fc94b53a39b37a729efbb69,1,kind_kaliouby


Let's try to join one of these domains. We start by indexing into it:

In [ ]:
pclient = opengrid[2]

In [11]:
pclient

(This is a logged out ProxyClient() object for a domain called 'kind_kaliouby'. Please call .login(email, password) to get a full client you can use for stuff.)

We then we login as normal, as we would do to any domain!

In [12]:
client = pclient.login(email="info@openmined.org", password="changethis")

Logged in to kind_kaliouby as info@openmined.org


Now we can view datasets, and use this domain node like we would any other:

In [18]:
client.datasets

In the event that we wanted our domain node to join a network, we could proceed as follows:

In [2]:
# Login to our local domain node
domain = sy.login(email="info@openmined.org", password="changethis", port=8081)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to None... done! 	 Logging into ecstatic_chollet... done!


In [17]:
# Applying to join is as simple as a one liner.
domain.apply_to_network(opengrid)

🔌 <DomainClient - kind_kaliouby: <UID: 201e5de97fc94b53a39b37a729efbb69>> successfully connected to the VPN: http://40.83.192.48:80/api/v1
Waiting to connect to VPN.
Connected to VPN
Application submitted.


# Summary
In this notebook, you created a new user, learned how to change the user permissions, file and respond to requests, and even join a network node!
In the next lesson, we'll be giving you a bigger taste of remote data science, and what you can do with it. Stay tuned!


<hr>
<hr>